### Markdown example
Blab bla documentation


In [3]:
import requests
import untangle
import pprint

pp = pprint.PrettyPrinter(indent=4)

r = requests.get('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=212403', auth=('user', 'pass'))
print r
print ''
print 'Status code: ' 
print r.status_code
print ''
print 'Headers: '
print r.headers
print ''


xmlObj = untangle.parse(r.text)

pmid_lst = []

for ref in xmlObj.pmc_articleset.article.back.ref_list.ref:
    this_cite = ref.citation
    
    if hasattr(this_cite, 'pub_id'):
        pubs = this_cite.pub_id
        if type(pubs) == list:
            for pub_el in pubs:
                if pub_el['pub-id-type'] == 'pmid':
                    #print 'Reference: '
                    #print this_cite.article_title.cdata
                    #print 'Entry type:'
                    #print type(pub_el)
                    #print 'Has PMID:'
                    #print pub_el.cdata
                    pmid_lst.append(pub_el.cdata)
        elif pubs['pub-id-type'] == 'pmid':
            #print 'Reference: '
            #print this_cite.article_title.cdata
            #print 'Entry type:'
            #print type(pubs)
            #print 'Has PMID:'
            #print pubs.cdata
            pmid_lst.append(pubs.cdata)
            
print 'All cited PMIDs:'
print pmid_lst
print len(pmid_lst)

<Response [200]>

Status code: 
200

Headers: 
{'X-XSS-Protection': '1; mode=block', 'NCBI-PHID': '03ED5B8C816250710000000003D303C0', 'Content-Security-Policy': 'upgrade-insecure-requests', 'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked', 'Set-Cookie': 'ncbi_sid=03ED5B8C81630B51_0979SID; domain=.nih.gov; path=/; expires=Mon, 30 Oct 2017 17:41:09 GMT', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'Vary': 'Accept-Encoding', 'Keep-Alive': 'timeout=1, max=10', 'Server': 'Apache', 'Content-Security-Policy-Report-Only': "default-src https:; script-src https: 'unsafe-inline' 'unsafe-eval'; font-src https: data:; img-src https: data:; style-src https: 'unsafe-inline'; report-uri https://www.ncbi.nlm.nih.gov/corecgi/csp/csp.cgi", 'NCBI-SID': '03ED5B8C81630B51_0979SID', 'Connection': 'Keep-Alive', 'X-UA-Compatible': 'IE=Edge', 'Cache-Control': 'private', 'Date': 'Sun, 30 Oct 2016 17:41:09 GMT', 'Access-Control-Allow-Origin': '*', 'Content-Type': 'text/x

In [3]:
import sys
sys.path.append('../web/lib')
from ConnectEutils import ConnectEutils
import pandas as pd


eutils = ConnectEutils()

pd.DataFrame(eutils.get_cited_PMID('212403'), columns=['PMID'])

ImportError: No module named pandas

In [6]:
import xml.etree.ElementTree as ET
import requests 
import urllib2

tree = ET.ElementTree(file=urllib2.urlopen('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=212403' ))
tree = ET.ElementTree(file=urllib2.urlopen('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=4807304' ))


pmid_lst = []

for element in tree.iter():
    if element.tag == 'pub-id':
        if element.attrib['pub-id-type'] == 'pmid':
            pmid_lst.append(element.text)
            
print pmid_lst


[]


In [13]:
import sys
sys.path.append('../web/lib')
from ConnectEutils import ConnectEutils


eutils = ConnectEutils()

pub1 = eutils.get_cited_PMID_etree(PMID="5055858")
pub2 = eutils.get_cited_PMID_etree(PMID="5107590")

#print pub1
#print pub2

print list(set(pub1) & set(pub2))

[]


## Simulate a browser session

In [4]:
import os
os.chdir('../backend')
import lib
import utils
from lib.GraphSession import GraphSession
from lib.ResultGraph import ResultGraph
from networkx import generate_gml

graphsession = GraphSession()
citations = graphsession.getCitationsFromPMIDString('27729734,25995680')
resultGraph = ResultGraph()
resultGraph.populate_from_cite_dict(citations)
res = resultGraph.get_cy_json()
print res['nodes']
#print generate_gml(res)
#print citations

TypeError: __init__() takes exactly 2 arguments (1 given)

In [3]:
import os
os.chdir('../backend')
#sys.path.append('../backend/lib')
#sys.path.append('../backend/lib')
import lib
import utils
from lib.GraphSession import GraphSession

## Create CyJSON output

{
    "nodes": [
            {
            "data": {"id": "a", "label": "Gene1"}
            },
            {
            "data": {"id": "b", "label": "Gene2"}
            },
            {
            "data": {"id": "c", "label": "Gene3"}
            },
            {
            "data": {"id": "d", "label": "Gene4"}
            },
            {
            "data": {"id": "e", "label": "Gene5"}
            },
            {
            "data": {"id": "f", "label": "Gene6"}
            }
    ],
            "edges": [
            {
            "data": {
            "id": "ab",
                    "source": "a",
                    "target": "b"
            }
            },
            {
            "data": {
            "id": "cd",
                    "source": "c",
                    "target": "d"
            }
            },
            {
            "data": {
            "id": "ef",
                    "source": "e",
                    "target": "f"
            }
            },
            {
            "data": {
            "id": "ac",
                    "source": "a",
                    "target": "d"
            }
            },
            {
            "data": {
             "id": "be",
                    "source": "b",
                    "target": "e"
                }
                }]    
    }

In [6]:
import os
os.chdir('../backend')
import lib
import utils
from lib.GraphSession import GraphSession
from lib.ResultGraph import ResultGraph
from networkx import generate_gml

graphsession = GraphSession('27729734,25995680')
citations = graphsession.getCitationsFromPMIDString('27729734,25995680')
resultGraph = ResultGraph()
resultGraph.populate_from_cite_dict(citations)
n_json = resultGraph.get_cy_json()

# Parse nodes
node_lst = []
id_lst = [] # Due to networkx' format, edges refer to nodes in the form of their list position, so we'll store their position here.
for node in n_json['nodes']:
    node_lst.append({'data': {'id':node['id'], 'name': node['id']}})
    id_lst.append(node['id'])
    
# Parse edges
edge_lst = []
for edge in n_json['links']:
    edge_lst.append({'data':{'id':'{a:d}_{b:d}'.format(a=edge['source'], b=edge['target']), 
                             'source':id_lst[edge['source']], 
                             'target':id_lst[edge['target']] }})
cy_dict = {'nodes': node_lst, 'edges': edge_lst}
print cy_dict

01/06/2018 04:03:14 PM [INFO]: RequestFullTextSearch: Creating request
01/06/2018 04:03:14 PM [INFO]: MongoSession: Mongo db connection successful
01/06/2018 04:03:14 PM [INFO]: ConnectEutils: Connecting to eutils API:
 https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pmc_pmc_cites&id=27729734&tool=GraphSearch
01/06/2018 04:03:16 PM [INFO]: ConnectEutils: Connecting to eutils API:
 https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pmc_pmc_cites&id=25995680&tool=GraphSearch


TypeError: string indices must be integers, not str

In [1]:
import sys
sys.path.append('../web/lib')
from GraphSession import GraphSession
from ResultGraph import ResultGraph
from networkx import generate_gml

graphsession = GraphSession()
citations = graphsession.getCitationsFromPMIDString('27729734,25995680')
resultGraph = ResultGraph()
resultGraph.populate_from_cite_dict(citations)
resultGraph.get_cy_json()
resultGraph.G

ImportError: No module named GraphSession

In [4]:
# Simulate a browser session using elink example
import sys
import json
from networkx.readwrite import json_graph
sys.path.append('../web/lib')
from GraphSession import GraphSession
from ResultGraph import ResultGraph

graphsession = GraphSession('24752654')
graphsession.parse_input()
graphsession.load_citations()
resultgraph = ResultGraph()
print graphsession.cite_dict
resultgraph.populate_from_cite_dict(graphsession.cite_dict)

this_graph = resultgraph.G

for n in this_graph:
    print n
data = json_graph.node_link_data(this_graph)
s = json.dumps(data)
print s
#resultgraph.get_json()
#27729734,27895396,27785449

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pubmed_refs&id=24752654&tool=GraphSearch
{'24752654': ['24752654', '24091329', '23746840', '23420871', '23386393', '23336100', '22918226', '22865453', '22777646', '22722893', '22635005', '22357162', '22318521', '22258609', '22216980', '21989406', '21876726', '21653254', '21640565', '21376230', '20884633', '20631206', '20628381', '20550684', '20179207', '20124221', '20019050', '20016485', '19603039', '19460998', '19381331', '19131956', '19033363', '18796163', '18775299', '18613948', '18589210', '18573355', '17656355', '17504872', '17486113', '17255514', '17109012', '16890165', '16651434', '16453000', '16102175', '14729627', '14679214', '13298683', '12716131', '12118079', '11872830', '11390481', '11248153', '10722846', '9563066', '8349690', '1905840', '1013510', '212572']}
1013510
20550684
22918226
22865453
18775299
12118079
20628381
19033363
8349690
11390481
14729627
18796163
17656355
16453000
2001648

## Try converting PMC ID to pmid

In [2]:
# Simulate a browser session using elink example
import sys
import json
from networkx.readwrite import json_graph
sys.path.append('../web/lib')
from GraphSession import GraphSession
from ResultGraph import ResultGraph

graphsession = GraphSession('24752654')
graphsession.parse_input()
graphsession.load_citations()
resultgraph = ResultGraph()
print graphsession.cite_dict
resultgraph.populate_from_cite_dict(graphsession.cite_dict)


https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pubmed_refs&id=24752654&tool=GraphSearch
eLinkResult
LinkSet
DbFrom
IdList
Id
24752654
LinkSetDb
DbTo
LinkName
Link
Id
24091329
Link
Id
23746840
Link
Id
23420871
Link
Id
23386393
Link
Id
23336100
Link
Id
22918226
Link
Id
22865453
Link
Id
22777646
Link
Id
22722893
Link
Id
22635005
Link
Id
22357162
Link
Id
22318521
Link
Id
22258609
Link
Id
22216980
Link
Id
21989406
Link
Id
21876726
Link
Id
21653254
Link
Id
21640565
Link
Id
21376230
Link
Id
20884633
Link
Id
20631206
Link
Id
20628381
Link
Id
20550684
Link
Id
20179207
Link
Id
20124221
Link
Id
20019050
Link
Id
20016485
Link
Id
19603039
Link
Id
19460998
Link
Id
19381331
Link
Id
19131956
Link
Id
19033363
Link
Id
18796163
Link
Id
18775299
Link
Id
18613948
Link
Id
18589210
Link
Id
18573355
Link
Id
17656355
Link
Id
17504872
Link
Id
17486113
Link
Id
17255514
Link
Id
17109012
Link
Id
16890165
Link
Id
16651434
Link
Id
16453000
Link
Id
16102175
Link
Id
14729627
Lin

In [2]:
# Simulate a browser session using elink example
import sys
import json
from networkx.readwrite import json_graph
sys.path.append('../web/lib')
from ConnectEutils import ConnectEutils

eutils = ConnectEutils()

PMC_ID = 'PMC4413964'

pmid = eutils.get_pmid_from_PMC(PMC_ID)
print pmid

cite_lst = eutils.get_cited_pub(PMC_ID)
print cite_lst

25918260
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pubmed_refs&id=25918260&tool=GraphSearch
['25918260', '24606128', '24576654', '24520944', '24415877', '24384685', '24350564', '24325272', '24293372', '24283303', '24255719', '24251922', '24232581', '24210956', '24172183', '24171787', '24155775', '24123097', '24117517', '24083739', '24081937', '24064753', '24056623', '24002499', '23982274', '23980084', '23980077', '23962244', '23945731', '23922471', '23884750', '23869689', '23840128', '23800278', '23796082', '23741309', '23737955', '23719667', '23660416', '23597497', '23565616', '23561638', '23547075', '23523578', '23483664', '23466316', '23456227', '23447460', '23426336', '23402625', '23400333', '23377952', '23375519', '23331661', '23326137', '23288584', '23269606', '23216217', '23213040', '23188531', '23178708', '23157398', '23145027', '23134976', '23099728', '23095613', '23079897', '23023009', '23001077', '22994906', '22965574', '22948836'

In [5]:
x = 'PMC4413964'
if 'PMC' in x:
    print 'yup'

yup


In [4]:
import xml.etree.ElementTree as ET
import requests 
import urllib2

tree = ET.ElementTree(file=urllib2.urlopen('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=212403' ))
tree = ET.ElementTree(file=urllib2.urlopen('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=4807304' ))
tree = ET.ElementTree(file=urllib2.urlopen('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=cancer&reldate=60&datetype=edat&retmax=100&usehistory=y'))

pmid_lst = []

for element in tree.iter():
    if element.tag == 'Id':
        pmid_lst.append(element.text)
            
print pmid_lst


['28618452', '28618451', '28618450', '28618431', '28618430', '28618420', '28618417', '28618402', '28618401', '28618400', '28618399', '28618398', '28618397', '28618396', '28618395', '28618394', '28618350', '28618345', '28618342', '28618326', '28618311', '28618305', '28618304', '28618256', '28618252', '28618241', '28618234', '28618224', '28618207', '28618205', '28618197', '28618188', '28618180', '28618167', '28618162', '28618158', '28618156', '28618147', '28618135', '28618133', '28618128', '28618124', '28618116', '28618108', '28618103', '28618089', '28618088', '28618084', '28618075', '28618074', '28618070', '28618063', '28618048', '28618035', '28618017', '28618012', '28618011', '28617999', '28617998', '28617988', '28617976', '28617974', '28617957', '28617955', '28617933', '28617920', '28617917', '28617912', '28617902', '28617891', '28617890', '28617887', '28617886', '28617885', '28617880', '28617871', '28617862', '28617852', '28617833', '28617822', '28617818', '28617804', '28617799', '28

### Test connectivity filtering

In [8]:
import networkx as nx
import json
from networkx.readwrite import json_graph

G = nx.path_graph(7)
G.add_edge(5,6)
graphs = list(nx.connected_component_subgraphs(G))
for graph in graphs:
    print json_graph.node_link_data(graph)

{'directed': False, 'graph': {'name': 'path_graph(4)'}, 'nodes': [{'id': 0}, {'id': 1}, {'id': 2}, {'id': 3}], 'links': [{'source': 0, 'target': 1}, {'source': 1, 'target': 2}, {'source': 2, 'target': 3}], 'multigraph': False}
{'directed': False, 'graph': {'name': 'path_graph(4)'}, 'nodes': [{'id': 5}, {'id': 6}], 'links': [{'source': 0, 'target': 1}], 'multigraph': False}


In [4]:
import networkx as nx
import json
from networkx.readwrite import json_graph

G = nx.path_graph(7)
for n,attrdict in G.node.items():
    print attrdict
#SG=nx.Graph( [ (u,v,d) for u,v,d in G.edges(data=True) if v['weight']>connectivity] )
#SG=G.subgraph( [n for n,attrdict in G.node.items() if attrdict['type'] == 'X' ] )

{}
{}
{}
{}
{}
{}
{}


In [1]:
# Simulate a browser session using elink example
import sys
import json
from networkx.readwrite import json_graph
sys.path.append('../web/lib')
from GraphSession import GraphSession
from ResultGraph import ResultGraph

userInput = 'cancer'
graphsession = GraphSession()
citations = graphsession.get_citations_from_fulltext(userInput)
print 'Citations:'
print citations
resultGraph = ResultGraph()
resultGraph.populate_from_cite_dict(citations)
resultGraph.extract_by_connectivity()
resultGraph.get_cy_json()


https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pmc&sort=relevance&term=cancer
<Element 'Id' at 0x7fd294b20590>
2907408
<Element 'Id' at 0x7fd294b205d0>
4339370
<Element 'Id' at 0x7fd294b20610>
5339024
<Element 'Id' at 0x7fd294b20650>
3769458
<Element 'Id' at 0x7fd294b20690>
3921623
<Element 'Id' at 0x7fd294b206d0>
4580396
<Element 'Id' at 0x7fd294b20710>
4568174
<Element 'Id' at 0x7fd294b20750>
4588097
<Element 'Id' at 0x7fd294b20790>
3295744
<Element 'Id' at 0x7fd294b207d0>
4569005
<Element 'Id' at 0x7fd294b20810>
3080029
<Element 'Id' at 0x7fd294b20850>
5421136
<Element 'Id' at 0x7fd294b20890>
5393518
<Element 'Id' at 0x7fd294b208d0>
3560853
<Element 'Id' at 0x7fd294b20910>
4581024
<Element 'Id' at 0x7fd294b20950>
4819002
<Element 'Id' at 0x7fd294b20990>
4567704
<Element 'Id' at 0x7fd294b209d0>
4840031
<Element 'Id' at 0x7fd294b20a10>
4528018
<Element 'Id' at 0x7fd294b20a50>
4232908
mongodb connection successful
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
to interface with the current application object in a way.  To solve
this set up an application context with app.app_context().  See the
documentation for more information.

In [3]:
resultGraph.extract_by_connectivity(connectivity=0)
resultGraph.G